In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from mpl_toolkits import mplot3d
%matplotlib inline
from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D
import time

In [2]:
random.seed(1852026)

In [3]:
df = pd.read_excel("PROJECTS/dataPoints.xlsx")
data = df.to_numpy()#change to matrix

You have to randomly split the data set into a training set, a validation set and a test set. Suggested
percentages for this split are 70 %, 15% and 15%, respectively, **but feel free to experiment
and change these numbers.**

Define train, test and validation set

In [4]:
def initializeParams(N):
    n=2
    n_y=1
    W = np.random.randn(N,n) 
    bias = np.random.randn(N,1) 
    V = np.random.randn(n_y,N) 
    a=pd.DataFrame(V.T)
    a[2]=W[:,0]
    a[3]=W[:,1]
    a[4]=bias
    omega=np.matrix(a)
    
 
   
    return W,bias,V,omega

In [5]:
data_train, data_rest = train_test_split(data, test_size=0.30)

In [6]:
data_test, data_validate = train_test_split(data_rest, test_size=0.50)

Create input vectors

In [7]:
X_train = np.transpose(data_train[:,0:2])
Y_train = np.transpose(data_train[:,2:])
X_validate = np.transpose(data_validate[:,0:2])
Y_validate = np.transpose(data_validate[:,2:])
X_test = np.transpose(data_test[:,0:2])
Y_test = np.transpose(data_test[:,2:])


n = X_train.shape[0] # Size of the input vector X
n_y = Y_train.shape[0] # Size of the input vector Y
P = X_train.shape[1] # Size of the sample
N = 10 # size of the hidden layer, # neurons, this is just for the moment to have some example

In [8]:
df.head()

,x1,x2,y
0,-1.262271,0.746859,0.461002
1,-0.804355,0.937081,0.617336
2,-1.805674,0.738389,-0.053984
3,-0.888805,0.061711,1.943678
4,-1.667562,-0.534543,2.127070


# Question 1. (Full minimization)

### MLP network

In [9]:
W,bias,V,omega=initializeParams(6)
ro=10**-5 #10-5 unti, 10-3
sigma=1
N=5

In [10]:
print(V,'\n')
print(W,'\n')
print(bias)

[[ 0.43849469  0.06912601  0.08928278  0.15155908 -0.73601249 -0.10472084]] 

[[ 1.08067898  0.68388825]
 [ 3.2765603   0.03512145]
 [ 0.08561585 -0.39218973]
 [-0.59941984  1.1970872 ]
 [-0.76038139 -0.40552083]
 [ 1.6745501   1.37888041]] 

[[ 0.10170382]
 [ 0.3341281 ]
 [ 0.11189055]
 [-0.45791113]
 [ 1.82330765]
 [ 0.14409858]]


In [11]:
class MLP(object):
    
    def __init__(self, ro, sigma, N):
        self.ro = ro
        self.sigma = sigma 
        self.N = N

    def set_N(self, x): 
        self.N = x 
    def get_N(self): 
        return self.N     
    
    
    def second_norm(self,omega):
        return np.linalg.norm(omega)**2 

    def activation_f(self,t,sigma=1):
        return (np.exp(2*sigma*t)-1)/(np.exp(2*sigma*t)+1)

    def predict(self,omega,X):  

        '''
        r=omega.flatten().shape[0]
        try:
            c=omega.flatten().shape[1]

            N=int(omega.shape[1]/4)

        except:
                N=int(omega.shape[0]/4)
                
        '''     
        N=self.N
        V=omega.T[:N].reshape(1,N)
        W=omega.T[N:N+2*N].reshape(N,2)

        bias=omega.T[N+2*N:].reshape(N,1)

        t=W.dot(X)-bias


        predicted_values=V.dot(self.activation_f(t,sigma))
        return predicted_values#, W,bias




    def reg_tr_error(self,omega,functionArgs):
        X=functionArgs[0]
        true=functionArgs[1]
        
        '''
        r=omega.flatten().shape[0]
        
        try:
            c=omega.flatten().shape[1]

            N=int(omega.shape[1]/4)

        except:
                N=int(omega.shape[0]/4)

        '''
        predicted=self.predict(omega,X)


        err=np.array(predicted)-true #err_all=np.sum(np.array(predicted)-true)**2
        err_all=err.dot(err.T)


        P=X.shape[1]
        return ((err_all)/(2*P)+self.ro*self.second_norm(omega)).item(0)
    

  #  def get_params(self):
  #      return self.ro, self.sigma, self.N
    
  

In [12]:
W, bias, V, omega=initializeParams(7)
ro=10**-5 #10-5 unti, 10-3
sigma=1
N=7

In [13]:
mlp=MLP(ro,sigma,7)
mlp.reg_tr_error(omega.flatten(),[X_train,Y_train])

1.4377043473238826

In [14]:
mlp.get_N()

7

In [15]:
mlp.reg_tr_error(omega.flatten(),[X_train,Y_train])

1.4377043473238826

In [16]:
res=minimize(mlp.reg_tr_error,omega.flatten(), args=[X_train,Y_train],method='L-BFGS-B')
print(res['fun'],'\n',res['x'])

0.035109118342736945 
 [-1.16275922e+01 -3.54894713e+00 -8.67902098e+00  2.24051857e+00
 -3.75265050e+00  2.81962698e+00 -2.61732684e+00 -1.18092772e-01
 -9.97126407e-01  1.04547311e+00 -3.37641823e-02 -3.36950197e-02
 -7.27197097e-01  2.31123757e+00  4.15762143e-02  1.09723692e-02
  2.58669930e+00  1.07120950e-01 -1.70941081e+00  2.51340256e-01
  3.39135491e+00 -1.06744423e+00 -2.72471876e-01  2.73059955e+00
  9.84338921e-01  1.13653889e+00 -1.90642139e+00 -3.80345555e+00]


In [17]:
res['x'].shape

(28,)

In [18]:
print(mlp.reg_tr_error(res['x'].flatten(),[X_validate,Y_validate,N]))
print(mlp.reg_tr_error(res['x'].flatten(),[X_test,Y_test,N]))

0.04570328222129813
0.02954968028939653


$$ E(\omega,\pi) = \frac{1}{2P}\sum_{p=1}^{P}(f(x^p)-y^p)^2+\rho \| \omega\|^2$$

Hyperparameters:
        
    the number of neurons N of the hidden layer
    the spread delta in the activation function g(t)
    the regularization parameter rho

## DRAFT VERSION OF THE ""HOMEMADE"" CROSS-VALIDATION METHOD

In [19]:
# the .fit method part is sort of missing cause choosing different configuratuons is needed


#repeated CROSS Validation

In [20]:
def second_norm_jac(omega): 
    return np.linalg.norm(omega)

def mse(true,predicted):
    return (np.sum(np.array(true-predicted)**2))/true.shape[1]


def calculate_test_err(cv_data,neurons,rho,sigma,omega):
    err=[]
    Mlp=MLP(rho,sigma,neurons)
    for i in range(10):
        X_train = np.transpose(np.matrix(cv_data)[:,0:2])
        Y_train = np.transpose(np.matrix(cv_data)[:,2:])
        #train on data train and data validate
        res=minimize(Mlp.reg_tr_error,omega.flatten(), args=[X_train,Y_train,N],method='L-BFGS-B')
        omega=res['x']
        #test on test data
        err.append(mse(Y_test,Mlp.predict(omega.flatten(),X_test)))
    return np.mean(err)

In [21]:
W,bias,V,omega=initializeParams(N=3)
omega.flatten()

matrix([[-0.42808959, -0.11676097, -0.0335933 , -0.21847242,  1.34483608,
         -0.59777008,  2.31068604,  0.34143696,  0.81758636,  0.30356997,
          0.00572397,  0.81135695]])

In [22]:
def fivefoldCV(params):
    '''
    params a list N,rho,sigma
    '''
    global X_train, X_validate, X_test, Y_train, Y_validate, Y_test 

    K=5   # SO 4 folds for training 1-validation for testing; they switch every time
    cv_data=np.concatenate((data_train,data_validate))
    np.random.shuffle(cv_data)
    indices=np.arange(0,255,51)# [  0,  51, 102, 153, 204]
    folds=[]



  #  X=X_train

  #  true=Y_train

    N=params[0]
    rho=params[1]#10**-5 #10-5 unti, 10-3
    sigma=params[2]#1

    W,bias,V,omega=initializeParams(N)

    mlp=MLP(rho,sigma,N)


    P=cv_data.shape[0]

    mlp.set_N(N)
    
    #print(mlp.get_N())

    val_err_mse=[]
    train_err_mse=[]
    fun=[]
    jac_norm=[]
    init_tr_err=[]

    #train_err={}
    #val_err={}

    data=cv_data.copy()


    res_df=pd.DataFrame(columns=['neurons','rho','sigma','init_trr_err','fun','err_tr','jac_norm','err_val','err_test'])
    for i in range(len(indices)):
        cv_data=data
        if i<4:
            l=[i for i in range(indices[i],indices[i+1])]
            #(VALIDATION fold) for testing
            validate_cv=cv_data[indices[i]:indices[i+1],:]

            #train folds together for training
            df=pd.DataFrame(cv_data)
            train_cv=df.drop(df.index[l])



            X_train = np.transpose(np.matrix(train_cv)[:,0:2])
            Y_train = np.transpose(np.matrix(train_cv)[:,2:])
            X_validate = np.transpose(validate_cv[:,0:2])
            Y_validate = np.transpose(validate_cv[:,2:])
            init_tr_err.append( mse(Y_train,mlp.predict(omega.flatten(),X_train)))

            #print(omega.shape)
        #CHOSEN OMEGA? ->Fitting of the model
            res=minimize(mlp.reg_tr_error,omega.flatten(), args=[X_train,Y_train],method='L-BFGS-B')
     #       for i in range(10):
     #           omega=res['x']
     #           res=minimize(mlp.reg_tr_error,omega.flatten(), args=[X_train,Y_train],method='L-BFGS-B')
            
            omega=res['x']
            fun.append(res['fun'])
            jac_norm.append(second_norm_jac(res['jac'].T))

        #    err_tr=mse(reg_tr_error(omega.flatten(),[X_train,Y_train]))
            err_tr=mse(Y_train,mlp.predict(omega.flatten(),X_train))
            err_val=mse(Y_validate,mlp.predict(omega.flatten(),X_validate))

            train_err_mse.append(err_tr)
            val_err_mse.append(err_val)
            
             
           # train_err_dict[i]=
          #  val_err_dict[i]=mse(Y_validate,predict(omega.flatten(),X_validate))


        else:
            #for the last element
            l=list([i for i in range(indices[i],255)])
            #(VALIDATION fold) for testing
            validate_cv=cv_data[indices[i]:,:]

            #train folds together for training
            df=pd.DataFrame(cv_data)
            train_cv=df.drop(df.index[l])
            init_tr_err.append( mse(Y_train,mlp.predict(omega.flatten(),X_train)))
            #CHOSEN OMEGA? ->Fitting of the model
            res=minimize(mlp.reg_tr_error,omega.flatten(), args=[X_train,Y_train],method='L-BFGS-B')
          #  for i in range(10):
          #      omega=res['x']
          #      res=minimize(mlp.reg_tr_error,omega.flatten(), args=[X_train,Y_train],method='L-BFGS-B')
            
            omega=res['x']
            fun.append(res['fun'])
            jac_norm.append(second_norm_jac(res['jac'].T))


    
            err_tr=mse(Y_train,mlp.predict(omega.flatten(),X_train))
       #     err_tr=mse(reg_tr_error(omega.flatten(),[X_train,Y_train]))
            err_val=mse(Y_validate,mlp.predict(omega.flatten(),X_validate))

            train_err_mse.append(err_tr)
            val_err_mse.append(err_val)

      #      train_err_dict_mse[i]=mse(Y_train,predict(omega.flatten(),X_train))
       #     val_err_dict_mse[i]=mse(Y_validate,predict(omega,X_validate))

    err_test=calculate_test_err(cv_data,N,rho,sigma,omega)
    res_df=res_df.append({'neurons':N,'rho':rho,'sigma':sigma,'fun':np.mean(fun),'init_trr_err':np.mean(init_tr_err),\
                              'err_tr':np.mean(train_err_mse),'jac_norm':np.mean(jac_norm),\
                              'err_val':np.mean(val_err_mse),'err_test':err_test},ignore_index=True )
    return res_df

In [23]:
fivefoldCV([3,0.0001,1])

,neurons,rho,sigma,init_trr_err,fun,err_tr,jac_norm,err_val,err_test
0,3.0,0.0001,1.0,0.656607,0.133992,0.248211,0.000216,0.27378,0.271318


In [ ]:
neurons_count=[20]#[2,7,11,17,23,27,33]
rho_values=[0.001,0.00001]
sigma_vals=[0.2,1]

all_poss_conf=[]
for i in neurons_count:
    for r in rho_values:
        for s in sigma_vals:
            all_poss_conf.append((i,r,s))

In [ ]:
from tqdm import tqdm
res_df=pd.DataFrame()
for conf in tqdm(all_poss_conf):
    res_df=res_df.append(fivefoldCV(conf))

In [ ]:
res_df

In [ ]:
res_df

In [ ]:
res_df.to_csv('cv_results.csv')

In [ ]:
rho=all_poss_conf[-2][1]
sigma=all_poss_conf[-2][2]
N=all_poss_conf[-2][0]
W,bias,V,omega=initializeParams(N)

mlp=MLP(rho,sigma,N)
cv_data=np.concatenate((data_train,data_validate))
X_train = np.transpose(np.matrix(cv_data)[:,0:2])
Y_train = np.transpose(np.matrix(cv_data)[:,2:])
res=minimize(mlp.reg_tr_error,omega.flatten(), args=[X_train,Y_train],method='L-BFGS-B')
omega_star=res['x']


Z = mlp.predict(omega_star,X_test)

#mlp.predict(omega_star,)

In [ ]:
#print(mlp.reg_tr_error(res['x'].flatten(),[X_validate,Y_validate,N]))
print(mlp.reg_tr_error(omega_star,[X_test,Y_test,N]))

In [ ]:
np.sum((Z-Y_test)**2)/45

In [ ]:
#TRAIN ERROR


#TEST ERROR

In [ ]:
def plotting(): 
 ##   rho=all_poss_conf[-2][1]
 ##   sigma=all_poss_conf[-2][2]
 ##   mlp=MLP(rho,sigma)
    
    fig = plt.figure(figsize=(70,40))
    ax = plt.axes(projection='3d')
   
  #  cv_data=np.concatenate((data_train,data_validate))
  #  X_train = np.transpose(np.matrix(cv_data)[:,0:2])
  #  Y_train = np.transpose(np.matrix(cv_data)[:,2:])
    
##    X_train = np.transpose(np.matrix(data)[:,0:2])
##    Y_train = np.transpose(np.matrix(data)[:,2:])
##
##    res=minimize(mlp.reg_tr_error,omega.flatten(), args=[X_train,Y_train],method='L-BFGS-B')
  #  for i in range(5):
  #      omega2=res['x']
  #      res=minimize(mlp.reg_tr_error,omega2.flatten(), args=[X_train,Y_train],method='L-BFGS-B')
            
    omega2=omega_star
    
 #   xy = np.mgrid[-2:2.002:0.05, -1:1.002:0.05].reshape(2,-1).T
    xy = np.mgrid[-1:1.002:0.05, -2:2.002:0.05].reshape(2,-1).T

    X = xy[:, 0].squeeze()
    Y = xy[:, 1].squeeze()
    
    XY=np.concatenate((X,Y)).reshape(2,X.shape[0]).T

    
    Z = mlp.predict(omega_star,XY.T)
    Z = Z.T.reshape(X.shape[0],)

    ax = fig.gca(projection='3d')
    ax.view_init(azim=30)
   # ax.set_zlim(-2,6)
    surf = ax.plot_trisurf(X, Y, Z, linewidth=0.010, antialiased=True,cmap='viridis')
    fig.savefig('11_MLP')
    plt.grid()
  #  return X.shape,Y.shape,Z.shape,X,Y,Z.T
 
    
   

    plt.show()

In [ ]:
def s(a,v):
    return a+v
    return a**v

In [ ]:
s(2,3)

In [ ]:
d={}
d['1']=[4]
d['1']+=[4]

In [ ]:
plotting()

# Q1  1.2

In [ ]:
def initializeParams_q12(N):
    n=2
    n_y=1
    idxx = list(np.random.choice(X_train.shape[1], N))
    C = X_train[:,idxx].T##np.random.randn(N,n)#X_train[:,idxx].T#
    #print(C.shape)
    
    V = np.random.randn(N,n_y) 
    a=pd.DataFrame(V)
    a[2]=C[:,0]
    a[3]=C[:,1]
    #a[4]=bias
    omega=np.matrix(a)
    
 
   
    return C,V,omega

In [ ]:
class RBF(object):
    
    def __init__(self, ro, sigma, N):
        self.ro = ro
        self.sigma = sigma 
        self.N = N

    def set_N(self, x): 
        self.N = x 
    def get_N(self): 
        return self.N     
    
    
    def second_norm(self,omega):
        return np.linalg.norm(omega)**2 

    
    def predict0(self,omega,X):  

        '''
        r=omega.flatten().shape[0]
        try:
            c=omega.flatten().shape[1]

            N=int(omega.shape[1]/4)

        except:
                N=int(omega.shape[0]/4)
                
        '''     
        N=self.N
        omega=omega.flatten().reshape((N,3))
        V = omega[:,0]
        C = omega[:,1:3]
        t1=np.apply_along_axis(np.linalg.norm, 1,X.T - C[0])#X.T - C1
        t = np.exp(-(t1/sigma**2)**2)
        S=t*V[0].item(0)
        for y in range(1,N):
            t1=np.apply_along_axis(np.linalg.norm, 1,X.T - C[y])#X.T - C1
            t = np.exp(-(t1/sigma**2)**2)
            S=S+t*V[y].item(0)
        
        #predicted_values=S
        return S#, W,bias
    def predict2(self,omega,X):  

        N=self.N
        sig = self.sigma
        omega=omega.flatten().reshape((N,3))
        V = omega[:,0]
        #X= X_train
        C1 = omega[:,1].T
        C2 = omega[:,2].T
        X1 = (X[0].T).reshape((X.shape[1],1))
        X2 = (X[1].T).reshape((X.shape[1],1))
        X1 = X1 - C1
        X2 = X2 - C2
        XX = -np.square(np.sqrt(np.square(X1)+np.square(X2))/sig)#(np.linalg.norm(X1,axis=1) + np.linalg.norm(X2,axis=1))
        XX= np.exp(XX)
        CX= XX.dot(V)
        CX= CX.reshape((X.shape[1],))
        
        return CX



    def reg_tr_error(self,omega,functionArgs):
        X=functionArgs[0]
        true=functionArgs[1]
        
        '''
        r=omega.flatten().shape[0]
        
        try:
            c=omega.flatten().shape[1]

            N=int(omega.shape[1]/4)

        except:
                N=int(omega.shape[0]/4)

        '''
        predicted=self.predict2(omega,X)


        err=np.array(predicted)-true #err_all=np.sum(np.array(predicted)-true)**2
        err_all=err.dot(err.T)


        P=X.shape[1]
        return ((err_all)/(2*P)+self.ro*self.second_norm(omega)).item(0)
    

  #  def get_params(self):
  #      return self.ro, self.sigma, self.N
    
  

In [ ]:
ro=0.0001
sigma=1
N=2
C,V,omega=initializeParams_q12(N)

rbf=RBF(ro,sigma,N)
cv_data=np.concatenate((data_train,data_validate))
X_train = np.transpose(np.matrix(cv_data)[:,0:2])
Y_train = np.transpose(np.matrix(cv_data)[:,2:])
RR=rbf.predict0(omega.flatten(),X_train)
Rr1= rbf.predict2(omega.flatten(),X_train)
#print(RR)
#print(Rr1)
print(Y_train.shape)
print(mse(Y_train,Rr1))
print(mse(Y_train,RR))

st = time.time()
res=minimize(rbf.reg_tr_error,omega.flatten(), args=[X_train,Y_train],method='CG')
omega_star=res['x']
print(time.time()-st)
print(res['fun'])

just checking

In [ ]:
ro=0.00001
sigma=1.111
N=2
rbf=RBF(ro,sigma,N)
C,V,omega=initializeParams_q12(N)
rbf.predict2(omega.flatten(),X_validate).shape

In [ ]:
V = omega[:,0]
X= X_train
#X=np.expand_dims(X, axis=1)
C1 = omega[:,1].T
C2 = omega[:,2].T

X1 = (X[0].T).reshape((X.shape[1],1))
X2 = (X[1].T).reshape((X.shape[1],1))
X1 = X1 - C1
X2 = X2 - C2
XX = -np.square(np.sqrt(np.square(X1)+np.square(X2))/1.2)#(np.linalg.norm(X1,axis=1) + np.linalg.norm(X2,axis=1))
XX= np.exp(XX)#(np.linalg.norm(X1,axis=1) + np.linalg.norm(X2,axis=1))
CX= XX.dot(V)

In [ ]:
omega=omega.flatten().reshape((N,3))
V = omega[:,0]
C = omega[:,1:3]
t1=np.apply_along_axis(np.linalg.norm, 1,X.T - C[0])#X.T - C1
t = np.exp(-(t1/1.2)**2)
S=t*V[0].item(0)
for y in range(1,N):
    t1=np.apply_along_axis(np.linalg.norm, 1,X.T - C[y])#X.T - C1
    t = np.exp(-(t1/1.2)**2)
    S=S+t*V[y].item(0)

In [ ]:
S.shape

In [ ]:
CX.shape

In [ ]:
Y_train.shape


In [ ]:
mse(Y_train,CX.flatten())#rbf.predict2(omega_star.flatten(),Xtest))

In [ ]:
mse(Y_train,S)#rbf.predict2(omega_star.flatten(),Xtest))

In [ ]:
def fivefoldCV_q12(params):
    '''
    params a list N,rho,sigma
    '''
    global X_train, X_validate, X_test, Y_train, Y_validate, Y_test 

    K=3   # SO 4 folds for training 1-validation for testing; they switch every time
    cv_data=np.concatenate((data_train,data_validate))
    np.random.shuffle(cv_data)
    indices=np.arange(0,255,51)# [  0,  51, 102, 153, 204]
    folds=[]



  #  X=X_train

  #  true=Y_train

    N=params[0]
    rho=params[1]#10**-5 #10-5 unti, 10-3
    sigma=params[2]#1

    C,V,omega=initializeParams_q12(N)

    rbf=RBF(rho,sigma,N)


    P=cv_data.shape[0]

    rbf.set_N(N)
    
    #print(mlp.get_N())

    val_err_mse=[]
    train_err_mse=[]
    fun=[]
    jac_norm=[]
    init=[]

    #train_err={}
    #val_err={}

    data=cv_data.copy()


    res_df=pd.DataFrame(columns=['neurons','rho','sigma','fun','init','err_tr','jac_norm','err_val','err_test'])
    for i in range(len(indices)):
        cv_data=data
        if i<4:
            l=[i for i in range(indices[i],indices[i+1])]
            #(VALIDATION fold) for testing
            validate_cv=cv_data[indices[i]:indices[i+1],:]

            #train folds together for training
            df=pd.DataFrame(cv_data)
            train_cv=df.drop(df.index[l])



            X_train = np.transpose(np.matrix(train_cv)[:,0:2])
            Y_train = np.transpose(np.matrix(train_cv)[:,2:])
            X_validate = np.transpose(validate_cv[:,0:2])
            Y_validate = np.transpose(validate_cv[:,2:])
            init.append(mse(Y_train,rbf.predict2(omega.flatten(),X_train)))

            #print(omega.shape)
        #CHOSEN OMEGA? ->Fitting of the model
            res=minimize(rbf.reg_tr_error,omega.flatten(), args=[X_train,Y_train],method='CG')
     #       for i in range(10):
     #           omega=res['x']
     #           res=minimize(mlp.reg_tr_error,omega.flatten(), args=[X_train,Y_train],method='L-BFGS-B')
            
            omega=res['x']
            fun.append(res['fun'])
            jac_norm.append(second_norm_jac(res['jac'].T))

        #    err_tr=mse(reg_tr_error(omega.flatten(),[X_train,Y_train]))
            err_tr=mse(Y_train,rbf.predict2(omega.flatten(),X_train))
            err_val=mse(Y_validate,rbf.predict2(omega.flatten(),X_validate))

            train_err_mse.append(err_tr)
            val_err_mse.append(err_val)
            
             
           # train_err_dict[i]=
          #  val_err_dict[i]=mse(Y_validate,predict(omega.flatten(),X_validate))


        else:
            #for the last element
            l=list([i for i in range(indices[i],255)])
            #(VALIDATION fold) for testing
            validate_cv=cv_data[indices[i]:,:]

            #train folds together for training
            df=pd.DataFrame(cv_data)
            train_cv=df.drop(df.index[l])
            init.append(mse(Y_train,rbf.predict2(omega.flatten(),X_train)))

            #CHOSEN OMEGA? ->Fitting of the model
            res=minimize(rbf.reg_tr_error,omega.flatten(), args=[X_train,Y_train],method='CG')
          #  for i in range(10):
          #      omega=res['x']
          #      res=minimize(mlp.reg_tr_error,omega.flatten(), args=[X_train,Y_train],method='L-BFGS-B')
            
            omega=res['x']
            fun.append(res['fun'])
            jac_norm.append(second_norm_jac(res['jac'].T))


    
            err_tr=mse(Y_train,rbf.predict2(omega.flatten(),X_train))
       #     err_tr=mse(reg_tr_error(omega.flatten(),[X_train,Y_train]))
            err_val=mse(Y_validate,rbf.predict2(omega.flatten(),X_validate))

            train_err_mse.append(err_tr)
            val_err_mse.append(err_val)

      #      train_err_dict_mse[i]=mse(Y_train,predict(omega.flatten(),X_train))
       #     val_err_dict_mse[i]=mse(Y_validate,predict(omega,X_validate))

    err_test=calculate_test_err(cv_data,N,rho,sigma,omega)
    res_df=res_df.append({'neurons':N,'rho':rho,'sigma':sigma,'fun':np.mean(fun),'init':np.mean(init),\
                              'err_tr':np.mean(train_err_mse),'jac_norm':np.mean(jac_norm),\
                              'err_val':np.mean(val_err_mse),'err_test':err_test},ignore_index=True )
    return res_df

In [ ]:
def second_norm_jac(omega): 
    return np.linalg.norm(omega)

def mse(true,predicted):
    return (np.sum(np.array(true-predicted)**2))/true.shape[1]


def calculate_test_err(cv_data,neurons,rho,sigma,omega):
    err=[]
    Rbf=RBF(rho,sigma,neurons)
    for i in range(10):
        X_train = np.transpose(np.matrix(cv_data)[:,0:2])
        Y_train = np.transpose(np.matrix(cv_data)[:,2:])
        #train on data train and data validate
        res=minimize(Rbf.reg_tr_error,omega.flatten(), args=[X_train,Y_train],method='CG')
        omega=res['x']
        #test on test data
        err.append(mse(Y_test,Rbf.predict2(omega.flatten(),X_test)))
    return np.mean(err)

In [ ]:
neurons_count=[2,9,17,25,33]
rho_values=[0.001,0.0001,0.00001]
sigma_vals=[1.11,1.455,1.99]

all_poss_conf=[]
for i in neurons_count:
    for r in rho_values:
        for s in sigma_vals:
            all_poss_conf.append((i,r,s))

def get_data():
    data = df.to_numpy()
    data_train, data_rest = train_test_split(data, test_size=0.30)
    data_test, data_val = train_test_split(data_rest, test_size=0.50)
    data_train, data_validate, data_test = np.split(df.sample(frac=1), [int(.7*len(df)), int(.85*len(df))])
    data_train = data_train.to_numpy()
    data_validate = data_validate.to_numpy()
    data_test = data_test.to_numpy()

    X_tr = np.transpose(data_train[:,0:2])
    Y_tr = np.transpose(data_train[:,2:])
    X_val = np.transpose(data_validate[:,0:2])
    Y_val = np.transpose(data_validate[:,2:])
    X_test = np.transpose(data_test[:,0:2])
    Y_test = np.transpose(data_test[:,2:])
    return X_tr,Y_tr,X_val,Y_val,X_test,Y_test

X, y_true, Xval,Yval, Xtest, Ytest = get_data()
funn,starting_err,ending_err,jac_norm,val_err,test_err=[],[],[],[],[],[]
conf_q2=pd.DataFrame(columns=['neurons','rho','sigma','fun','start_err'\
                                            ,'ending_err','jac_norm',\
                                           'val_err','test_err'])
for conf in tqdm(all_poss_conf):
    C,V,omega=initializeParams_q12(conf[0])

    rbf=RBF(conf[1],conf[2],conf[0])
    res=minimize(rbf.reg_tr_error,omega.flatten(), args=[X,y_true],method='L-BFGS-B')
    omega_star=res['x']
    funn=res['fun']
    #pred_i = rbf.predict(omega.flatten(),X)
    starting_err=mse(y_true,rbf.predict2(omega.flatten(),X))
    ending_err=mse(y_true,rbf.predict2(omega_star.flatten(),X))
    jac_norm=second_norm_jac(res['jac'].T)
    val_err=mse(Yval,rbf.predict(omega_star.flatten(),Xval))
    X=np.concatenate((X.T,Xval.T)).T
    y_true=np.concatenate((y_true.T,Yval.T)).T
    res=minimize(rbf.reg_tr_error,omega_star.flatten(), args=[X,y_true],method='L-BFGS-B')
    omega_star=res['x']
    test_err=mse(Ytest,rbf.predict2(omega_star.flatten(),Xtest))
    conf_q2=conf_q2.append({'neurons':conf[0],'rho':conf[1],'sigma':conf[2],'fun':funn,'start_err':starting_err,\
                              'ending_err':funn,'jac_norm':jac_norm,\
                              'val_err':val_err,'test_err':test_err},ignore_index=True )


conf_q2

In [ ]:
fivefoldCV_q12([3,0.0001,1])

In [ ]:
#from tqdm.notebook import tqdm


In [ ]:
from tqdm import tqdm
res_df=pd.DataFrame()
for conf in tqdm(all_poss_conf):
    res_df=res_df.append(fivefoldCV_q12(conf))

In [ ]:
res_df.to_csv('q12_trial1.csv')

In [ ]:
res_df

In [ ]:
neurons_count=[50,88,125]
rho_values=[0.001,0.0001,0.00001]
sigma_vals=[1.11,1.455,1.99]

all_poss_conf=[]
for i in neurons_count:
    for r in rho_values:
        for s in sigma_vals:
            all_poss_conf.append((i,r,s))

In [ ]:
from tqdm.notebook import tqdm
res_df=pd.DataFrame()
for conf in tqdm(all_poss_conf):
    res_df=res_df.append(fivefoldCV_q12(conf))

In [ ]:
res_df.to_csv('q12_trial1_bigvals.csv')

# Q2 Extreme Learning 2.1

In [ ]:
class MLP_Q2(object):
    
    def __init__(self, ro, sigma, N):
        self.ro = ro
        self.sigma = sigma 
        self.N = N

    def set_N(self, x): 
        self.N = x 
    def get_N(self): 
        return self.N     
    
    
    def second_norm(self,omega):
        return np.linalg.norm(omega)**2 

    def activation_f(self,t,sigma=1):
        return (np.exp(2*sigma*t)-1)/(np.exp(2*sigma*t)+1)

    def predict(self,X, W, bias, V):  
  
        N=self.N
        t=W.dot(X)-bias
        #print(t.shape)
        predicted_values=V.dot(self.activation_f(t,sigma))
        return predicted_values#, W,bias




    def reg_tr_error(self,V,functionArgs):
        X=functionArgs[0]
        true=functionArgs[1]
        W = functionArgs[2]
        bias = functionArgs[3]
        
        predicted=self.predict(X,W,bias,V)


        err=np.array(predicted)-true #err_all=np.sum(np.array(predicted)-true)**2
        err_all=err.dot(err.T)


        P=X.shape[1]
        #omega_upd=np.append(np.append(V,W, axis=1), bias, axis=1)
        a=pd.DataFrame(V.T)
        a[2]=W[:,0]
        a[3]=W[:,1]
        a[4]=bias
        omega_upd=np.matrix(a)
        return ((err_all)/(2*P)+self.ro*self.second_norm(omega_upd)).item(0)
    

  #  def get_params(self):
  #      return self.ro, self.sigma, self.N
    
  

In [ ]:
def initializeParams_q2(N):
    n=2
    n_y=1
    W = np.random.randn(N,n)*np.random.randn(N,n)*np.random.randn(N,n)
    bias = np.random.randn(N,1)*np.random.randn(N,1)*np.random.randn(N,1)
    V = np.random.randn(n_y,N) 
    a=pd.DataFrame(V.T)
    a[2]=W[:,0]
    a[3]=W[:,1]
    a[4]=bias
    omega=np.matrix(a)
    
 
   
    return W,bias,V,omega

In [ ]:
from scipy import optimize
ro=0.00001#all_poss_conf[-2][1]
sigma=1#all_poss_conf[-2][2]
N=13#all_poss_conf[-2][0]
W,bias,V,omega=initializeParams_q2(N)

mlp_q2=MLP_Q2(ro,sigma,N)
cv_data=np.concatenate((data_train,data_validate))
X_train = np.transpose(np.matrix(cv_data)[:,0:2])
Y_train = np.transpose(np.matrix(cv_data)[:,2:])


In [ ]:
res=minimize(mlp_q2.reg_tr_error,V, args=[X_train,Y_train, W, bias],method='CG')


In [ ]:
V_star=res['x']

In [ ]:
res

In [ ]:
print(mlp_q2.reg_tr_error(V_star,[X_train,Y_train,W,bias]))

In [ ]:
print(mlp_q2.reg_tr_error(V_star,[X_test,Y_test,W,bias]))

In [ ]:
def q_2_mlp(rho,sigma,N,rep, X,Y,X_test,Y_test):
    test_err=[]
    train_err=[]
    from tqdm import tqdm
    for r in tqdm(range(rep)):
        W,bias,V,omega=initializeParams_q2(N)
        mlp=MLP_Q2(rho,sigma,N)
        res=minimize(mlp.reg_tr_error,V, args=[X,Y, W, bias],method='CG')
        V_star=res['x']
        curr_err=mlp.reg_tr_error(V_star,[X_test,Y_test,W,bias])
        test_err.append(curr_err)
        train_err.append(mlp.reg_tr_error(V_star,[X,Y,W,bias]))
        if np.min(test_err)==curr_err:
            best = (W,bias,V_star)
    return best,test_err,train_err

In [ ]:
best_V_W_b,test_errs,train_errs=q_2_mlp(ro,sigma,N,10, X_train,Y_train,X_test,Y_test)

In [ ]:
test_errs

In [ ]:
train_errs

In [ ]:
WW=best_V_W_b[0]
res_q2=pd.DataFrame(WW[:,0])
res_q2[2]=WW[:,1]
res_q2[3]=best_V_W_b[1]
res_q2[4]=best_V_W_b[2]
res_q2.to_csv('res_q2_W_b_V.csv')


In [ ]:
res_q2=pd.read_csv('res_q2_W_b_V.csv',usecols=[1,2,3,4])
omega_upd=res_q2

In [ ]:
omega_upd.values

In [ ]:
best_V_W_b

In [ ]:
mlp_q2=MLP_Q2(ro,sigma,N)
print(mlp_q2.reg_tr_error(best_V_W_b[2],[X_test,Y_test,best_V_W_b[0],best_V_W_b[1]]))

In [ ]:
def plotting_q2(W_inp,bias_inp,V_inp): 
 ##   rho=all_poss_conf[-2][1]
 ##   sigma=all_poss_conf[-2][2]
 ##   mlp=MLP(rho,sigma)
    
    fig = plt.figure(figsize=(70,40))
    ax = plt.axes(projection='3d')
   
  #  cv_data=np.concatenate((data_train,data_validate))
  #  X_train = np.transpose(np.matrix(cv_data)[:,0:2])
  #  Y_train = np.transpose(np.matrix(cv_data)[:,2:])
    
##    X_train = np.transpose(np.matrix(data)[:,0:2])
##    Y_train = np.transpose(np.matrix(data)[:,2:])
##
##    res=minimize(mlp.reg_tr_error,omega.flatten(), args=[X_train,Y_train],method='L-BFGS-B')
  #  for i in range(5):
  #      omega2=res['x']
  #      res=minimize(mlp.reg_tr_error,omega2.flatten(), args=[X_train,Y_train],method='L-BFGS-B')
            
    #omega2=omega_star
    a=pd.DataFrame(V_inp.T)
    a[2]=W_inp[:,0]
    a[3]=W_inp[:,1]
    a[4]=bias_inp
    omega2=np.matrix(a)
    
 #   xy = np.mgrid[-2:2.002:0.05, -1:1.002:0.05].reshape(2,-1).T
    xy = np.mgrid[-1:1.002:0.05, -2:2.002:0.05].reshape(2,-1).T

    X = xy[:, 0].squeeze()
    Y = xy[:, 1].squeeze()
    
    XY=np.concatenate((X,Y)).reshape(2,X.shape[0]).T
    #print(XY)

    
    Z = mlp_q2.predict(XY.T,W_inp,bias_inp,V_inp)
    Z = Z.T.reshape(X.shape[0],)

    ax = fig.gca(projection='3d')
    ax.view_init(azim=30)
   # ax.set_zlim(-2,6)
    surf = ax.plot_trisurf(X, Y, Z, linewidth=0.010, antialiased=True,cmap='viridis')
    fig.savefig('11_MLP')
    plt.grid()
  #  return X.shape,Y.shape,Z.shape,X,Y,Z.T
 
    
   

    plt.show()

In [ ]:
uu=omega_upd.values[:,0:2]
bb=omega_upd.values[:,2].reshape(N,1)
vv=omega_upd.values[:,3].reshape(N,1).T
mlp_q2=MLP_Q2(ro,sigma,N)
print(mlp_q2.reg_tr_error(vv,[X_test,Y_test,np.array(uu),bb]))

In [ ]:
plotting_q2(np.array(uu),bb,vv)# with 13 neurons

In [ ]:
plotting_q2(np.array(uu),bb,vv)# with 25 neurons

In [ ]:
plotting_q2(np.array(uu),bb,vv)# with 20 neurons

In [ ]:
plotting_q2(np.array(uu),bb,vv)# with 33 neurons

In [ ]:
plotting_q2(np.array(uu),bb,vv)# with 23 neurons

In [ ]:
plotting_q2(best_V_W_b[0],best_V_W_b[1],best_V_W_b[2]) # with 17 neurons

In [ ]:
plotting_q2(best_V_W_b[0],best_V_W_b[1],best_V_W_b[2]) # with 3 neurons

### Q2 Unsupervised selection of centers 2.2

In [96]:
## Choose centers

In [97]:
import itertools
from sklearn.cluster import KMeans

In [98]:
class RBF_Q2(object):
    
    def __init__(self, ro, sigma, N):
        self.ro = ro
        self.sigma = sigma 
        self.N = N

    def set_N(self, x): 
        self.N = x 
    def get_N(self): 
        return self.N     
    
    
    def second_norm(self,omega):
        return np.linalg.norm(omega)**2 

    
    def predict0(self,omega,X):  

        '''
        r=omega.flatten().shape[0]
        try:
            c=omega.flatten().shape[1]

            N=int(omega.shape[1]/4)

        except:
                N=int(omega.shape[0]/4)
                
        '''     
        N=self.N
        omega=omega.flatten().reshape((N,3))
        V = omega[:,0]
        C = omega[:,1:3]
        t1=np.apply_along_axis(np.linalg.norm, 1,X.T - C[0])#X.T - C1
        t = np.exp(-(t1/sigma**2)**2)
        S=t*V[0].item(0)
        for y in range(1,N):
            t1=np.apply_along_axis(np.linalg.norm, 1,X.T - C[y])#X.T - C1
            t = np.exp(-(t1/sigma**2)**2)
            S=S+t*V[y].item(0)
        
        #predicted_values=S
        return S#, W,bias
    def predict2(self,omega,X):  

        N=self.N
        sig = self.sigma
        omega=omega.flatten().reshape((N,3))
        V = omega[:,0]
        #X= X_train
        C1 = omega[:,1].T
        C2 = omega[:,2].T
        X1 = (X[0].T).reshape((X.shape[1],1))
        X2 = (X[1].T).reshape((X.shape[1],1))
        X1 = X1 - C1
        X2 = X2 - C2
        XX = -np.square(np.sqrt(np.square(X1)+np.square(X2))/sig)#(np.linalg.norm(X1,axis=1) + np.linalg.norm(X2,axis=1))
        XX= np.exp(XX)
        CX= XX.dot(V)
        CX= CX.reshape((X.shape[1],))
        
        return CX

    def predict22(self,V,C,X):  

        N=self.N
        sig = self.sigma
        V=V.flatten().reshape((N,1))
        #X= X_train
        C1 = C[:,0].T
        C2 = C[:,1].T
        X1 = (X[0].T).reshape((X.shape[1],1))
        X2 = (X[1].T).reshape((X.shape[1],1))
        X1 = X1 - C1
        X2 = X2 - C2
        XX = -np.square(np.sqrt(np.square(X1)+np.square(X2))/sig)#(np.linalg.norm(X1,axis=1) + np.linalg.norm(X2,axis=1))
        XX= np.exp(XX)
        CX= XX.dot(V)
        CX= CX.reshape((X.shape[1],))
        
        return CX


    def reg_tr_error(self,V,functionArgs):
        X=functionArgs[0]
        true=functionArgs[1]
        C=functionArgs[2]

        V = V.flatten().reshape((self.N,1))
        
        '''
        r=omega.flatten().shape[0]
        
        try:
            c=omega.flatten().shape[1]

            N=int(omega.shape[1]/4)

        except:
                N=int(omega.shape[0]/4)

        '''
        predicted=self.predict22(V,C,X)


        err=np.array(predicted)-true #err_all=np.sum(np.array(predicted)-true)**2
        err_all=err.dot(err.T)


        P=X.shape[1]
        return ((err_all)/(2*P)+self.ro*self.second_norm(omega)).item(0)
    

  #  def get_params(self):
  #      return self.ro, self.sigma, self.N
    
  

In [128]:
ini = 10 #number of repetitions to reduce the beginnings with "bad points"
N = 50
rho = 0.00001
sigma = 1.11

In [121]:
def choose_centers(X,N,ini=2):
    for _ in range(ini):
    # define the random initial parameters
        v_init = [random.uniform(0, 1) for i in range(N)]
        
        kmeans=KMeans(n_clusters=N, random_state=_)
        a=kmeans.fit(X.T)
        c_init=a.cluster_centers_
    return c_init

In [122]:
def initializeParams_q22(N,X):
    n_y=1
    V = np.random.randn(N,n_y) # Maybe we should choose some small values (0,1)????
    C = choose_centers(X,N,10)
    
    a=pd.DataFrame(V)
    a[2]=C[:,0]
    a[3]=C[:,1]
    omega=np.matrix(a)
    
    return V,C,omega

In [123]:
V,C,omega = initializeParams_q22(N,X_train)

Gradient of the regularized error:
$$\nabla_{v} E(v,c)=\frac{1}{P}\Phi ^{T}(\Phi v \;- \;Y)\;+\;2 \rho v$$

def grad_reg_error(V, args):
    
    X = args[0]
    Y = args[1]
    C0 = args[2]
    N = args[3]
    rho = args[4]
    sigma = args[5]
    
    
    V=V.flatten().reshape((N,1))
    
    #V = omega[:,0]
    #C1 = omega[:,1].T
    #C2 = omega[:,2].T
    C1 = C0[:,0].T
    C2 = C0[:,1].T
    X1 = (X[0].T).reshape((X.shape[1],1))
    X2 = (X[1].T).reshape((X.shape[1],1))
    X1 = X1 - C1
    X2 = X2 - C2
    XX = -np.square(np.sqrt(np.square(X1)+np.square(X2))/sigma)
    Phi= np.exp(XX)
    grad = (1/X.shape[1])*((Phi.T).dot((Phi.dot(V))-Y.T))+2*rho*V
    
    return grad.flatten()

In [103]:
def second_norm_jac(omega): 
    return np.linalg.norm(omega)

def mse(true,predicted):
    return (np.sum(np.array(true-predicted)**2))/true.shape[1]


def calculate_test_err(cv_data,C,neurons,rho,sigma,V):
    err=[]
    Rbf=RBF_Q2(rho,sigma,neurons)
    for i in range(10):
        X_train = np.transpose(np.matrix(cv_data)[:,0:2])
        Y_train = np.transpose(np.matrix(cv_data)[:,2:])
        #train on data train and data validate
        res=minimize(Rbf.reg_tr_error,V.flatten(), args=[X_train,Y_train,C,neurons,rho,sigma],method='CG',jac=grad_reg_error)
        V=res['x']
        #test on test data
        err.append(mse(Y_test,Rbf.predict22(V.flatten(),C,X_test)))
    return np.mean(err)

In [138]:
def fivefoldCV_q22(params):
    '''
    params a list N,rho,sigma
    '''
    global X_train, X_validate, X_test, Y_train, Y_validate, Y_test 

    K=5   # SO 4 folds for training 1-validation for testing; they switch every time
    cv_data=np.concatenate((data_train,data_validate))
    np.random.shuffle(cv_data)
    indices=np.arange(0,255,51)# [  0,  51, 102, 153, 204]
    folds=[]



  #  X=X_train

  #  true=Y_train

    N=params[0]
    rho=params[1]#10**-5 #10-5 unti, 10-3
    sigma=params[2]#1

    V,C0,omega = initializeParams_q22(N,X_train)

    rbf=RBF_Q2(rho,sigma,N)


    P=cv_data.shape[0]

    rbf.set_N(N)
    
    #print(mlp.get_N())

    val_err_mse=[]
    train_err_mse=[]
    fun=[]
    jac_norm=[]
    init=[]

    #train_err={}
    #val_err={}

    data=cv_data.copy()


    res_df=pd.DataFrame(columns=['neurons','rho','sigma','fun','init','err_tr','jac_norm','err_val'])
    for i in range(len(indices)):
        cv_data=data
        if i<4:
            l=[i for i in range(indices[i],indices[i+1])]
            #(VALIDATION fold) for testing
            validate_cv=cv_data[indices[i]:indices[i+1],:]

            #train folds together for training
            df=pd.DataFrame(cv_data)
            train_cv=df.drop(df.index[l])



            X_train = np.transpose(np.matrix(train_cv)[:,0:2])
            Y_train = np.transpose(np.matrix(train_cv)[:,2:])
            X_validate = np.transpose(validate_cv[:,0:2])
            Y_validate = np.transpose(validate_cv[:,2:])
            init.append(mse(Y_train,rbf.predict22(V,C,X_train)))

            #print(omega.shape)
        #CHOSEN OMEGA? ->Fitting of the model
            res=minimize(rbf.reg_tr_error,V, args=[X_train,Y_train,C,N,rho,sigma],method='CG',jac=grad_reg_error)
     #       for i in range(10):
     #           omega=res['x']
     #           res=minimize(mlp.reg_tr_error,omega.flatten(), args=[X_train,Y_train],method='L-BFGS-B')
            
            V=res['x']
            fun.append(res['fun'])
            jac_norm.append(second_norm_jac(res['jac'].T))

        #    err_tr=mse(reg_tr_error(omega.flatten(),[X_train,Y_train]))
            err_tr=mse(Y_train,rbf.predict22(V,C,X_train))
            err_val=mse(Y_validate,rbf.predict22(V,C,X_validate))

            train_err_mse.append(err_tr)
            val_err_mse.append(err_val)
            
             
           # train_err_dict[i]=
          #  val_err_dict[i]=mse(Y_validate,predict(omega.flatten(),X_validate))


        else:
            #for the last element
            l=list([i for i in range(indices[i],255)])
            #(VALIDATION fold) for testing
            validate_cv=cv_data[indices[i]:,:]

            #train folds together for training
            df=pd.DataFrame(cv_data)
            train_cv=df.drop(df.index[l])
            init.append(mse(Y_train,rbf.predict22(V,C,X_train)))

            #CHOSEN OMEGA? ->Fitting of the model
            res=minimize(rbf.reg_tr_error,V, args=[X_train,Y_train,C,N,rho,sigma],method='CG',jac=grad_reg_error)
          #  for i in range(10):
          #      omega=res['x']
          #      res=minimize(mlp.reg_tr_error,omega.flatten(), args=[X_train,Y_train],method='L-BFGS-B')
            
            V=res['x']
            fun.append(res['fun'])
            jac_norm.append(second_norm_jac(res['jac'].T))


    
            err_tr=mse(Y_train,rbf.predict22(V,C,X_train))
       #     err_tr=mse(reg_tr_error(omega.flatten(),[X_train,Y_train]))
            err_val=mse(Y_validate,rbf.predict22(V,C,X_validate))

            train_err_mse.append(err_tr)
            val_err_mse.append(err_val)

      #      train_err_dict_mse[i]=mse(Y_train,predict(omega.flatten(),X_train))
       #     val_err_dict_mse[i]=mse(Y_validate,predict(omega,X_validate))

   # err_test=calculate_test_err(cv_data,N,rho,sigma,omega,V)
    res_df=res_df.append({'neurons':N,'rho':rho,'sigma':sigma,'fun':np.mean(fun),'init':np.mean(init),\
                              'err_tr':np.mean(train_err_mse),'jac_norm':np.mean(jac_norm),\
                              'err_val':np.mean(val_err_mse)},ignore_index=True )
    return res_df

In [105]:
N

33

In [106]:
## Trying to do optimization

In [107]:
pars

[33, 1e-05, 1.11]

In [129]:
pars =[N,rho,sigma]
V,C,omega = initializeParams_q22(N,X_train)
args=[X_train,Y_train,C,N,rho,sigma]
rbf = RBF_Q2(rho,sigma,N)

In [130]:
X = args[0]
Y = args[1]
C0 = args[2]
N = args[3]
rho = args[4]
sigma = args[5]

In [110]:
N

33

In [131]:
a=rbf.reg_tr_error(V.flatten(),args)

In [132]:
a # starting value of the error

1.32063238690182

In [113]:
def grad_reg_error(V, args):
    try:
        X = args[0]
        Y = args[1]
        C0 = args[2]
        N = args[3]
        rho = args[4]
        sigma = args[5]
    
    
        V=V.flatten().reshape((N,1))

        #V = omega[:,0]
        #C1 = omega[:,1].T
        #C2 = omega[:,2].T
        C1 = C0[:,0].T
        C2 = C0[:,1].T
   
        X1 = (X[0].T).reshape((X.shape[1],1))
        X2 = (X[1].T).reshape((X.shape[1],1))
  
        X1 = X1 - C1
        X2 = X2 - C2
#        print(4)

        XX = -np.square(np.sqrt(np.square(X1)+np.square(X2))/sigma)
#        print(5)


        Phi= np.exp(XX)
#        print('phi',Phi.shape)
#        print('rho',rho)
#        print('V',V.shape)
#        print('dssa',((Phi.dot(V))-Y.T).shape) #204x1
#        print('xx',(((Phi.T).dot((Phi.dot(V))-Y.T))).shape)
#        print('da',V.shape[0])
#        print((1/X.shape[1])*(((Phi.T).dot((Phi.dot(V))-Y.T))).flatten())#.reshape((X.shape[1],1))
        
##        print((2*rho*V).shape)
        #        grad = (1/X.shape[1])*((Phi.T).dot((Phi.dot(V))-Y.T))+2*rho*V
        grad1=((1/X.shape[1])*(((Phi.T).dot((Phi.dot(V))-Y.T))))#.flatten()
  ##      print('ds',grad1.shape)
        gg=(2*rho*V)#.reshape((X.shape[1],1))
##        print('1',grad1.flatten()[0],'shape',grad1.shape)
##        print('2',gg.shape)
        grad=np.sum([grad1,gg],axis=0)#.flatten()
   #     grad = (1/X.shape[1])*((Phi.T).dot((Phi.dot(V))-Y.T))+2*rho*V
    except:
        print('mjau')  
    return grad.flatten()

In [114]:
def grad_reg_error(V, args):
    try:
        X = args[0]
        Y = args[1]
        C0 = args[2]
        N = args[3]
        rho = args[4]
        sigma = args[5]
    
    
        V=V.flatten().reshape((N,1))

        #V = omega[:,0]
        #C1 = omega[:,1].T
        #C2 = omega[:,2].T
        C1 = C0[:,0].T
        C2 = C0[:,1].T
   
        X1 = (X[0].T).reshape((X.shape[1],1))
        X2 = (X[1].T).reshape((X.shape[1],1))
  
        X1 = X1 - C1
        X2 = X2 - C2

        XX = -np.square(np.sqrt(np.square(X1)+np.square(X2))/sigma)

        

        Phi= np.exp(XX)
#     

        #        grad = (1/X.shape[1])*((Phi.T).dot((Phi.dot(V))-Y.T))+2*rho*V
        grad1=((1/X.shape[1])*(((Phi.T).dot((Phi.dot(V))-Y.T))))#.flatten()
  ##      print('ds',grad1.shape)
        gg=(2*rho*V)#.reshape((X.shape[1],1))
##        print('1',grad1.flatten()[0],'shape',grad1.shape)
##        print('2',gg.shape)
        grad=np.sum([grad1,gg],axis=0)#.flatten()
   #     grad = (1/X.shape[1])*((Phi.T).dot((Phi.dot(V))-Y.T))+2*rho*V
    except:
        print('mjau')
    return grad.flatten()

In [115]:
#grad_reg_error(V.flatten(), args=[X_train,Y_train,C,N,rho,sigma])

In [136]:
res=minimize(rbf.reg_tr_error,V.flatten(), args=[X_train,Y_train,C,N,rho,sigma],method='CG',jac=grad_reg_error)

In [137]:
res

     fun: 0.053020115066551125
     jac: array([-1.12041289e-06, -7.44488737e-06, -3.69582877e-06, -3.95945790e-06,
       -5.85806771e-06, -5.04973567e-06, -6.01939641e-06, -5.53130864e-06,
       -5.24325675e-06, -4.06520036e-06, -2.38299882e-06, -2.32199015e-06,
        1.16617505e-06, -6.28756970e-06, -3.77748347e-06, -1.35692066e-06,
       -5.84517078e-06, -3.15820168e-06, -9.42642978e-07, -2.82503674e-06,
       -3.78161514e-06, -9.20720951e-06, -5.89192842e-06, -1.50626249e-06,
       -1.90224835e-06, -3.52442183e-06, -4.14778213e-06, -4.20650620e-06,
       -6.62789194e-06, -6.59730898e-06, -1.79433312e-06,  7.04749128e-07,
       -4.42027983e-06, -1.81368593e-06, -2.97886625e-06, -3.20856292e-06,
        9.60467321e-08, -6.82393216e-06, -2.50283498e-06, -2.64603514e-06,
        1.10112380e-06,  5.29948263e-07, -1.39659505e-06,  2.83866204e-07,
        2.86350337e-06, -3.90047880e-06, -6.60342374e-06, -6.62992561e-06,
       -6.28829882e-06, -3.96396433e-06])
 message: 'Optimi

In [118]:
params = [N,rho,sigma]
res = fivefoldCV_q22(params)

In [119]:
res

,neurons,rho,sigma,fun,init,err_tr,jac_norm,err_val,err_test
0,33.0,0.00001,1.11,0.063565,2.470723,0.125164,0.002681,0.185102,NaN
